# How to convert tools to OpenAI Functions

This notebook goes over how to use LangChain tools as OpenAI functions.

```{=mdx}
import Npm2Yarn from "@theme/Npm2Yarn";

<Npm2Yarn>
  @langchain/community @langchain/openai
</Npm2Yarn>
```

In [16]:
import { Calculator } from "@langchain/community/tools/calculator";
import { HumanMessage } from "@langchain/core/messages";
import { convertToOpenAIFunction } from "@langchain/core/utils/function_calling";
import { ChatOpenAI } from "@langchain/openai";

In [17]:
const model = new ChatOpenAI({
  model: "gpt-3.5-turbo"
});

In [18]:
const tools = [new Calculator()]
const functions = [convertToOpenAIFunction(tools[0])]

In [19]:
functions[0]

{
  name: "calculator",
  description: "Useful for getting the result of a math expression. The input to this tool should be a valid mathema"... 63 more characters,
  parameters: {
    type: "object",
    properties: { input: { type: "string" } },
    additionalProperties: false,
    "$schema": "http://json-schema.org/draft-07/schema#"
  }
}

In [20]:
const modelWithTools= model.bind({
  functions,
})

In [21]:
const message = await modelWithTools.invoke(
    [new HumanMessage("use a calculator to add 2 and 3")]
)

In [22]:
message

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {
      function_call: { name: "calculator", arguments: '{"input":"2 + 3"}' },
      tool_calls: undefined
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: { name: "calculator", arguments: '{"input":"2 + 3"}' },
    tool_calls: undefined
  },
  response_metadata: {
    tokenUsage: { completionTokens: 17, promptTokens: 77, totalTokens: 94 },
    finish_reason: "function_call"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [23]:
message.additional_kwargs["function_call"]

{ name: "calculator", arguments: '{"input":"2 + 3"}' }

Or we can use the update OpenAI API that uses `tools` and `tool_choice` instead of `functions` and `function_call` by using `ChatOpenAI.bind({ tools })`:

In [24]:
const modelWithTools = model.bind({
  tools
})
await modelWithTools.invoke([new HumanMessage("use a calculator to add 2 and 3")])

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    tool_calls: [
      {
        name: "calculator",
        args: { input: "2 + 3" },
        id: "call_R2WXRRIbgh2qn1XBBu8P91DH"
      }
    ],
    invalid_tool_calls: [],
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_R2WXRRIbgh2qn1XBBu8P91DH",
          type: "function",
          function: [Object]
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_R2WXRRIbgh2qn1XBBu8P91DH",
        type: "function",
        function: { name: "calculator", arguments: '{"input":"2 + 3"}' }
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 17, promptTokens: 77, totalTokens: 94 },
    finish_reason: "tool_calls"
  },
  tool_calls: [
    {
      name: "calculator"